In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from collections import Counter
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFR
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df = pd.read_csv("spam_ham_dataset.csv")
df.replace({r'\r\n':' '}, regex=True, inplace=True)
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,3624,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs this deal is to b...,0


In [33]:
df.isna().sum()

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64

In [34]:
df.shape

(5171, 4)

In [35]:
df.duplicated().sort_values(ascending=False)

0       False
3474    False
3452    False
3451    False
3450    False
        ...  
1722    False
1721    False
1720    False
1719    False
5170    False
Length: 5171, dtype: bool

In [36]:
ps = PorterStemmer()
corpus = []

all_stop_words = set(stopwords.words('english'))
all_stop_words.remove('not')

for i in range (len(df)):
    text = df['text'][i].lower().translate(str.maketrans('','', string.punctuation)).split()
    text = [ps.stem(word) for word in text if word not in all_stop_words]
    text = ' '.join(text)
    corpus.append(text)

In [37]:
mostCommon = Counter(corpus).most_common(20)


In [38]:
cv = CountVectorizer(max_features= 42500)
X = cv.fit_transform(corpus).toarray()
y = df['label_num']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [40]:
def model_score(y_true,y_pred):
    acc_scor = accuracy_score(y_true, y_pred)
    prec_scor = precision_score(y_true, y_pred)
    recall_scor = recall_score(y_true, y_pred)
    f1_scor = f1_score(y_true, y_pred)
    overall_avg_score = (acc_scor + prec_scor + recall_scor + f1_scor) / 4

    print(f'Model accuracy score: {acc_scor}')
    print(f'Model precision score: {prec_scor}')
    print(f'Model recall score: {recall_scor}')
    print(f'Model f1 score: {f1_scor}')
    print(f'Average overall score performance: {overall_avg_score}')

    print(confusion_matrix(y_true, y_pred))

In [41]:
cl_rf = RFR(n_estimators=100, random_state=42)
cl_rf.fit(X_train, y_train)
y_pred = cl_rf.predict(X_test)

In [42]:
np.column_stack((y_test[:15], y_pred[:15]))

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

In [43]:
model_score(y_test,y_pred)


Model accuracy score: 0.9719806763285024
Model precision score: 0.9506578947368421
Model recall score: 0.9537953795379538
Model f1 score: 0.9522240527182867
Average overall score performance: 0.9571645008303963
[[717  15]
 [ 14 289]]
